In [2]:
import sys
import gc
import joblib
sys.path.append('../../')

from scoring.event_detection_matrix import competition_score
from models.rnn import LightningModel

from sklearn.preprocessing import StandardScaler

import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

In [3]:
checkpoint_path = "../../models/wandb/checkpoints/rnn_1_1024/model.ckpt"
model = LightningModel.load_from_checkpoint(checkpoint_path)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

cuda


LightningModel(
  (model): PyTorchRNN(
    (rnn): LSTM(2, 1024, batch_first=True)
    (fc): Linear(in_features=1024, out_features=2, bias=True)
    (softmax): Softmax(dim=1)
  )
  (train_metrics): MetricCollection(
    (MulticlassAccuracy): MulticlassAccuracy()
    (MulticlassPrecision): MulticlassPrecision()
    (MulticlassRecall): MulticlassRecall()
    (MulticlassF1Score): MulticlassF1Score(),
    prefix=train_
  )
  (val_metrics): MetricCollection(
    (MulticlassAccuracy): MulticlassAccuracy()
    (MulticlassPrecision): MulticlassPrecision()
    (MulticlassRecall): MulticlassRecall()
    (MulticlassF1Score): MulticlassF1Score(),
    prefix=val_
  )
)

In [34]:
df_validation_events = pd.read_csv('../../data/processed/validation_events_split.csv')

In [35]:
df_validation_events = df_validation_events[df_validation_events.step.notnull()]
df_validation_events

,series_id,night,event,step,timestamp,num_series_id
0,062dbd4c95e6,1,onset,7872.0,2018-08-22T23:11:00-0400,7
1,062dbd4c95e6,1,wakeup,14484.0,2018-08-23T08:22:00-0400,7
6,062dbd4c95e6,4,onset,60720.0,2018-08-26T00:35:00-0400,7
7,062dbd4c95e6,4,wakeup,68400.0,2018-08-26T11:15:00-0400,7
8,062dbd4c95e6,5,onset,77304.0,2018-08-26T23:37:00-0400,7
...,...,...,...,...,...,...
2955,fcca183903b7,33,wakeup,565824.0,2019-04-28T06:52:00-0400,276
2956,fcca183903b7,34,onset,577344.0,2019-04-28T22:52:00-0400,276
2957,fcca183903b7,34,wakeup,584052.0,2019-04-29T08:11:00-0400,276
2958,fcca183903b7,35,onset,595344.0,2019-04-29T23:52:00-0400,276


In [7]:
LAGS_FUTURE = [f"t_lag_{i}" for i in range(-1, -25, -1)]
LAGS_PAST = reversed([f"t_lag_{i}" for i in range(1, 25)])
FEATURES = [*LAGS_PAST, 't_0', *LAGS_FUTURE]

In [8]:
smoothing_length = 12 * 30 # 30 Minutes

def get_events_smoothed(test_series) :
    series_ids = test_series['num_series_id'].unique()
    events = []

    for idx in tqdm(series_ids):
        # Collecting sample and normalizing features
        X = test_series[test_series.num_series_id == idx]
                
        # We average the confidence, that the participant is awake
        X["confidence_awake"] = X["prediction_confidence_1"].rolling(smoothing_length, center=True).mean().bfill().ffill()
        X["asleep"] = X["prediction_confidence_0"].rolling(smoothing_length, center=True).mean().bfill().ffill()

        # Binarize the asleep column
        X["asleep"] = X["asleep"].round()

        # Getting predicted onset and wakeup time steps
        pred_onsets = X[X['asleep'].diff() > 0]['step'].tolist() # diff is > 0 if it changes from 0 (awake) to 1 (asleep)
        pred_wakeups = X[X['asleep'].diff() < 0]['step'].tolist() # diff is < 0 if it changes from 1 (asleep) to 0 (awake)
     
        if len(pred_onsets) > 0:

            # Ensuring all predicted sleep periods begin and end
            if min(pred_wakeups) < min(pred_onsets):
                pred_wakeups = pred_wakeups[1:]

            if max(pred_onsets) > max(pred_wakeups):
                pred_onsets = pred_onsets[:-1]

            # Keeping sleep periods longer than 30 minutes
            sleep_periods = [(onset, wakeup) for onset, wakeup in zip(pred_onsets, pred_wakeups) if wakeup - onset >= 12 * 30]

            for onset, wakeup in sleep_periods:
                # We take the score
                score = 1 - X[(X['step'] >= onset) & (X['step'] < wakeup)]['confidence_awake'].mean()

                # Adding sleep event to dataframe
                onset_row = {'row_id': len(events), 'series_id': idx, 'step': onset, 'event': 'onset', 'score': score}                
                events.append(onset_row)

                wakeup_row = {'row_id': len(events), 'series_id': idx, 'step': wakeup, 'event': 'wakeup', 'score': score}
                events.append(wakeup_row)

    return pd.DataFrame(events)

In [9]:
def prediction_batch(batch):
    X = batch
    
    with torch.no_grad():
        logits = model(X)
    
    label = torch.argmax(logits, dim=-1)
    confidence = torch.softmax(logits, dim=-1)
    confidence_0 = confidence[:, 0]
    confidence_1 = confidence[:, 1]
    return label, confidence_0, confidence_1

In [18]:
def prediction(overview_info, series):
    predictions = overview_info[['num_series_id', 'step']]
    
    label_list = []
    confidence_0_list = []
    confidence_1_list = []
    
    # series_length, series_columns = series[FEATURES].values.shape

    dataset = TensorDataset(series)
    dataloader = DataLoader(dataset, batch_size=1024)
    for index, batch in enumerate(tqdm(dataloader)): 
        label, confidence_0, confidence_1 = prediction_batch(batch[0])
        
        label_list.append(label)
        confidence_0_list.append(confidence_0)
        confidence_1_list.append(confidence_1) 
        
    predictions['prediction_class'] = torch.cat(label_list).cpu().numpy()
    predictions['prediction_confidence_0'] = torch.cat(confidence_0_list).cpu().numpy()
    predictions['prediction_confidence_1'] = torch.cat(confidence_1_list).cpu().numpy()

    return predictions

In [29]:
overview = pd.read_parquet('../../data/processed/transformer/validation/overview.parquet')
events_list = []

series_ids = overview.num_series_id.unique()

for i, num_series_id in enumerate(series_ids):
    print(f'Step {i+1} of {len(series_ids)}')
    current_series = overview[overview['num_series_id'] == num_series_id]
    series_prepared = torch.load('../../data/processed/transformer/validation/' + str(num_series_id) + '.pt').to(device)
    predictions = prediction(current_series, series_prepared)
    events = get_events_smoothed(predictions)

    events_list.append(events)

    del series_prepared
    del predictions
    gc.collect()

events = pd.concat(events_list).reset_index(drop=True)

Step 1 of 54


100%|██████████| 727/727 [01:02<00:00, 11.59it/s]
/tmp/ipykernel_5866/1180831845.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions['prediction_class'] = torch.cat(label_list).cpu().numpy()
100%|██████████| 1/1 [00:00<00:00, 13.17it/s]


Step 2 of 54


100%|██████████| 270/270 [00:22<00:00, 11.85it/s]
/tmp/ipykernel_5866/1180831845.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions['prediction_class'] = torch.cat(label_list).cpu().numpy()
100%|██████████| 1/1 [00:00<00:00, 33.99it/s]


Step 3 of 54


...

100%|██████████| 394/394 [00:33<00:00, 11.70it/s]
/tmp/ipykernel_5866/1180831845.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions['prediction_class'] = torch.cat(label_list).cpu().numpy()
100%|██████████| 1/1 [00:00<00:00, 22.99it/s]


Step 53 of 54


100%|██████████| 203/203 [00:16<00:00, 12.03it/s]
/tmp/ipykernel_5866/1180831845.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions['prediction_class'] = torch.cat(label_list).cpu().numpy()
100%|██████████| 1/1 [00:00<00:00, 52.26it/s]


Step 54 of 54


100%|██████████| 607/607 [00:52<00:00, 11.60it/s]
/tmp/ipykernel_5866/1180831845.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions['prediction_class'] = torch.cat(label_list).cpu().numpy()
100%|██████████| 1/1 [00:00<00:00, 15.51it/s]


In [37]:
df_validation_events['series_id'] = df_validation_events['num_series_id']

In [38]:
competition_score(df_validation_events, events)

0.39066575069275605

In [47]:
events.to_csv('./data/predictions_1024.csv', index=False)